# Raw Data

- [***Download Link***](https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36)

> ***`OBS_ASOS_TIM_data0`*** : 2021-01-01 ~ 2021-12-31 
>
> ***`OBS_ASOS_TIM_data1`*** : 2022-01-01 ~ 2023-01-01


- 2021-01-01부터 2023-01-01 약 2년간의 데이터셋

- 00시부터 23시까지 1시간 간격으로 측정한 데이터셋 (08시부터 18시까지 관측된 데이터가 대다수)

- 총 49개의 지점

In [105]:
import pandas as pd
import numpy as np
import gc

In [106]:
df0 = pd.read_csv('./OBS_ASOS_TIM_data0.csv', encoding='cp949') # 2021-01-01 ~ 2021-12-31
df1 = pd.read_csv('./OBS_ASOS_TIM_data1.csv', encoding='cp949') # 2022-01-01 ~ 2023-01-01

In [107]:
df_raw = pd.concat([df0, df1])
df_raw.head()

,지점,지점명,일시,일사(MJ/m2)
0,93,북춘천,2021-01-01 08:00,0.00
1,93,북춘천,2021-01-01 09:00,0.37
2,93,북춘천,2021-01-01 10:00,0.96
3,93,북춘천,2021-01-01 11:00,1.40
4,93,북춘천,2021-01-01 12:00,1.72


In [160]:
df_raw.shape

(445248, 6)

In [109]:
# 2021-01-01 ~ 2023-01-01 dataset (2 years)
# df_raw.to_csv('./raw.csv')

# 전처리 내용

|기준변수명|처리내용|비고|
|:---|:---|:---|
|일시|일자 변수로부터 Datetime, hour 변수 생성|Datetime 일자 변수를 날짜형으로 변환(YYYY-MM-DD), hour(시간만)|
|지점명|제천지점에서 측정된 관측치($1$개) 제거|지점명과 지점은 동일|
|지점명|파주, 상주, 동두천 지점은 2021-10-12일 이전의 기록은 보유하고 있지 않음.|파주, 상주, 동두천 데이터 제거|
|지점명|충주는 2021-11-18일 이전의 기록은 보유하고 있지 않음.|충주 데이터 제거|
|hour(파생)|08~18시까지 필터링|05시부터 21시까지 기록이 일부 있긴 함.|

## 데이터 탐색 (for preprocessing)

In [110]:
gc.collect()

19

### 관측지점 수 

In [147]:
# 관측지점 수
len(df0['지점'].unique()), len(df1['지점'].unique())

(48, 49)

22년도에 관측지점이 하나 더 추가된 듯하다..

In [93]:
set(df1['지점명'].unique()) - (set(df0['지점명'].unique()) & set(df1['지점명'].unique()))

{'제천'}

In [96]:
df1[df1['지점명']=='제천']  # 관측기록이 1회밖에 없다.

,지점,지점명,일시,일사(MJ/m2)
167811,221,제천,2022-10-27 07:00,0.0


제천에서 관측된 값이 하나밖에 없으므로 제거해주는 것이 좋을 듯 하다.

In [78]:
df0['지점명'].unique(),  df1['지점명'].unique()

(array(['북춘천', '철원', '동두천', '파주', '대관령', '춘천', '백령도', '북강릉', '강릉', '서울',
        '인천', '원주', '울릉도', '수원', '충주', '서산', '청주', '대전', '추풍령', '안동', '상주',
        '포항', '대구', '전주', '창원', '광주', '부산', '목포', '여수', '흑산도', '고창', '홍성',
        '제주', '고산', '진주', '고창군', '영광군', '김해시', '순창군', '북창원', '양산시', '보성군',
        '강진군', '의령군', '함양군', '광양시', '청송군', '경주시'], dtype=object),
 array(['북춘천', '철원', '동두천', '파주', '대관령', '춘천', '백령도', '북강릉', '강릉', '서울',
        '인천', '원주', '울릉도', '수원', '충주', '서산', '청주', '대전', '추풍령', '안동', '상주',
        '포항', '대구', '전주', '창원', '광주', '부산', '목포', '여수', '흑산도', '고창', '홍성',
        '제주', '고산', '진주', '제천', '고창군', '영광군', '김해시', '순창군', '북창원', '양산시',
        '보성군', '강진군', '의령군', '함양군', '광양시', '청송군', '경주시'], dtype=object))

### 날짜형 변환 및 Datetime, hour 변수 생성

In [117]:
# dtype 변환 및 hour변수 생성.
df_raw['일시'] = pd.to_datetime(df_raw['일시'])
df_raw['Datetime'] = df_raw['일시'].dt.date
df_raw['hour'] = df_raw['일시'].dt.hour
df_raw.head()

,지점,지점명,일시,일사(MJ/m2),Datetime,hour
0,93,북춘천,2021-01-01 08:00:00,0.00,2021-01-01,8
1,93,북춘천,2021-01-01 09:00:00,0.37,2021-01-01,9
2,93,북춘천,2021-01-01 10:00:00,0.96,2021-01-01,10
3,93,북춘천,2021-01-01 11:00:00,1.40,2021-01-01,11
4,93,북춘천,2021-01-01 12:00:00,1.72,2021-01-01,12


In [148]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445248 entries, 0 to 230204
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   지점         445248 non-null  int64         
 1   지점명        445248 non-null  object        
 2   일시         445248 non-null  datetime64[ns]
 3   일사(MJ/m2)  445248 non-null  float64       
 4   Datetime   445248 non-null  object        
 5   hour       445248 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 23.8+ MB


In [126]:
df_raw.shape[0]/df_raw['지점명'].nunique() # 지점하나당 9000개 정도의 관측치가 있음.

9086.69387755102

### 시간대별 관측치 수 

In [179]:
df_raw['hour'].value_counts().sort_index()

0       114
1       114
2       114
3       114
4       114
5       186
6     12896
7     23300
8     33812
9     33827
10    33828
11    33811
12    33816
13    33821
14    33821
15    33820
16    33831
17    33807
18    33785
19    22851
20    13032
21      206
22      114
23      114
Name: hour, dtype: int64

00시부터 23시까지 필터링해서 홈페이지에서 다운받아왔지만 대다수 데이터가 08시~18시까지 관측되어있음.

### 지점별 관측치 수 (2021-01-01 ~ 2023-01-01)

In [127]:
pd.DataFrame(df_raw['지점명'].value_counts())

,지점명
백령도,9684
울릉도,9619
창원,9606
의령군,9605
북강릉,9603
강진군,9602
제주,9602
대구,9602
목포,9598
북창원,9597


In [149]:
365*2*11 # 11시간관측을 기준으로 했을 때 지점당 관측치 개수..

8030

대다수 지점에서 2년간 약 9500건 관측되었지만, 파주, 상주, 동두천, 충주, 제천(1)은 다른지점에 비해 관측치가 적음.

### 파주, 상주, 동두천

In [130]:
df_raw.head() ## 북춘천

,지점,지점명,일시,일사(MJ/m2),Datetime,hour
0,93,북춘천,2021-01-01 08:00:00,0.00,2021-01-01,8
1,93,북춘천,2021-01-01 09:00:00,0.37,2021-01-01,9
2,93,북춘천,2021-01-01 10:00:00,0.96,2021-01-01,10
3,93,북춘천,2021-01-01 11:00:00,1.40,2021-01-01,11
4,93,북춘천,2021-01-01 12:00:00,1.72,2021-01-01,12


In [139]:
df_raw[df_raw['지점명']=='파주'].iloc[[0]] 

,지점,지점명,일시,일사(MJ/m2),Datetime,hour
10961,99,파주,2021-10-12,0.0,2021-10-12,0


In [140]:
df_raw[df_raw['지점명']=='상주'].iloc[[0]]

,지점,지점명,일시,일사(MJ/m2),Datetime,hour
84525,137,상주,2021-10-12,0.0,2021-10-12,0


In [141]:
df_raw[df_raw['지점명']=='동두천'].iloc[[0]]

,지점,지점명,일시,일사(MJ/m2),Datetime,hour
9583,98,동두천,2021-10-12,0.0,2021-10-12,0


### 충주

In [142]:
df_raw[df_raw['지점명']=='충주'].iloc[[0]]

,지점,지점명,일시,일사(MJ/m2),Datetime,hour
60308,127,충주,2021-11-18 18:00:00,0.0,2021-11-18,18


### 제천

In [134]:
df_raw[df_raw['지점명']=='제천']

,지점,지점명,일시,일사(MJ/m2),Datetime,hour
167811,221,제천,2022-10-27 07:00:00,0.0,2022-10-27,7


> 보유기간 1904년 4월 ~ 현재 (지점별 상이함) 

- 파주, 상주, 동두천 지점은 2021-10-12일 이전의 기록은 보유하고 있지 않음.
- 충주는 2021-11-18일 이전의 기록은 보유하고 있지 않음.
- 제천은 2022-10-27일 단 하루만의 기록만 있음.

- ref: https://minwon.kma.go.kr/main/obvStn.do
- ref: https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36

---

## 데이터 처리 및 csv파일로 내보내기

### 제천, 파주, 상주, 동두천 제거

In [161]:
df_prep1 = df_raw[~df_raw['지점명'].str.contains('제천|파주|상주|동두천|충주')]

### 관측시간 08시 ~ 18시

In [174]:
df_prep1[(df_prep1['hour']>=8) & (df_prep1['hour'] <= 18)]

,지점,지점명,일시,일사(MJ/m2),Datetime,hour
0,93,북춘천,2021-01-01 08:00:00,0.00,2021-01-01,8
1,93,북춘천,2021-01-01 09:00:00,0.37,2021-01-01,9
2,93,북춘천,2021-01-01 10:00:00,0.96,2021-01-01,10
3,93,북춘천,2021-01-01 11:00:00,1.40,2021-01-01,11
4,93,북춘천,2021-01-01 12:00:00,1.72,2021-01-01,12
...,...,...,...,...,...,...
230200,283,경주시,2023-01-01 14:00:00,1.62,2023-01-01,14
230201,283,경주시,2023-01-01 15:00:00,1.30,2023-01-01,15
230202,283,경주시,2023-01-01 16:00:00,0.82,2023-01-01,16
230203,283,경주시,2023-01-01 17:00:00,0.29,2023-01-01,17


In [178]:
df_prep1[(df_prep1['hour']>=8) & (df_prep1['hour'] <= 18)]['hour'].value_counts().sort_index()

8     32069
9     32082
10    32080
11    32064
12    32069
13    32072
14    32073
15    32072
16    32083
17    32061
18    32038
Name: hour, dtype: int64

In [182]:
df_prep2 = df_prep1[(df_prep1['hour']>=8) & (df_prep1['hour'] <= 18)]
df_prep2.head()

,지점,지점명,일시,일사(MJ/m2),Datetime,hour
0,93,북춘천,2021-01-01 08:00:00,0.00,2021-01-01,8
1,93,북춘천,2021-01-01 09:00:00,0.37,2021-01-01,9
2,93,북춘천,2021-01-01 10:00:00,0.96,2021-01-01,10
3,93,북춘천,2021-01-01 11:00:00,1.40,2021-01-01,11
4,93,북춘천,2021-01-01 12:00:00,1.72,2021-01-01,12


In [185]:
df_prep2['지점명'].nunique()

44

In [187]:
df_prep2['지점명'].value_counts()

북춘천    8041
북강릉    8041
철원     8041
강진군    8041
백령도    8040
의령군    8040
홍성     8040
목포     8039
창원     8039
춘천     8038
제주     8037
원주     8037
대구     8037
서울     8036
북창원    8034
여수     8034
함양군    8034
대관령    8033
포항     8032
보성군    8032
인천     8030
청송군    8030
서산     8030
김해시    8029
고산     8029
경주시    8029
수원     8025
광양시    8023
안동     8023
순창군    8023
고창군    8022
청주     8020
추풍령    8018
전주     8017
영광군    8012
진주     8010
부산     8000
양산시    7998
고창     7992
울릉도    7985
광주     7936
강릉     7931
흑산도    7907
대전     7898
Name: 지점명, dtype: int64

In [186]:
df_prep2.to_csv('./prep_data.csv')